In [7]:
from bokeh.io import output_notebook, show
import bokeh.plotting
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.events import SelectionGeometry
import numpy as np

import panel as pn

bokeh.io.output_notebook()

Loading BokehJS ...

In [8]:
# Synthetic data
x = np.linspace(0, 30, 2000)
rng = np.random.default_rng(0)
y = (2.0*np.exp(-0.5*((x-8.5)/0.6)**2)
     + 1.2*np.exp(-0.5*((x-15.0)/1.0)**2)
     + 1.8*np.exp(-0.5*((x-22.0)/0.8)**2)
     + 0.12*rng.normal(size=x.size))

src = ColumnDataSource(dict(x=x, y=y))
peak_src = ColumnDataSource(dict(x=[], y=[]))

In [9]:

p = bokeh.plotting.figure(
    title="Box select a range; red dot marks tallest point in range",
    height=360, 
    width=840,
    x_axis_label="time", 
    y_axis_label="intensity",
    tools="pan,wheel_zoom,box_zoom,box_select,reset,save,hover",
    active_drag="box_select",  # selection UX
)
p.line("x","y", source=src, line_width=2)

# invisible selection helpers; suppress selection visuals
p.scatter(
    "x",
    "y", 
    source=src, 
    size=3,
    alpha=0.001, 
    line_alpha=0.001,
    nonselection_fill_alpha=0.0, 
    nonselection_line_alpha=0.0,
    selection_fill_alpha=0.0, 
    selection_line_alpha=0.0,
    hover_fill_alpha=0.0, 
    hover_line_alpha=0.0,
)

# red marker
p.scatter("x","y", source=peak_src, size=10, color="red")

GlyphRenderer(id='p1168', ...)

In [11]:
cb = CustomJS(args=dict(src=src, peak_src=peak_src), code="""
    const geom = cb_obj.geometry || {};
    const x0 = geom.x0, x1 = geom.x1;
    // clear old marker
    peak_src.data = {x:[], y:[]};

    if (x0 == null || x1 == null) {
        peak_src.change.emit();
        return;
    }
    const xmin = Math.min(x0, x1), xmax = Math.max(x0, x1);
    const x = src.data.x, y = src.data.y;

    let best_i = -1, best_y = -Infinity;
    for (let i = 0; i < x.length; i++) {
        if (x[i] >= xmin && x[i] <= xmax) {
            if (y[i] > best_y) { best_y = y[i]; best_i = i; }
        }
    }
    if (best_i >= 0) {
        peak_src.data = {x:[x[best_i]], y:[y[best_i]]};
        peak_src.change.emit();
    }

    // Clear selection highlight immediately so the plot doesn't look selected
    try { src.selected.indices = []; } catch(_) {}
""")

# Use the event class, not a string
p.js_on_event(SelectionGeometry, cb)

show(p)


In [12]:
import numpy as np
import bokeh
import bokeh.io
bokeh.io.output_notebook()

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.events import SelectionGeometry

# --- synthetic demo data ---
x = np.linspace(0, 30, 2000)
rng = np.random.default_rng(0)
y = (2.0*np.exp(-0.5*((x-8.5)/0.6)**2)
     + 1.2*np.exp(-0.5*((x-15.0)/1.0)**2)
     + 1.8*np.exp(-0.5*((x-22.0)/0.8)**2)
     + 0.05*rng.normal(size=x.size))

src = ColumnDataSource(dict(x=x, y=y))
# polygon for filled AUC
auc_src = ColumnDataSource(dict(px=[], py=[]))

p = figure(
    title="Box select a range; AUC of tallest peak is filled (red)",
    height=360, width=840,
    x_axis_label="time", y_axis_label="intensity",
    tools="pan,wheel_zoom,box_zoom,box_select,reset,save,hover",
    active_drag="box_select",
)
p.line("x","y", source=src, line_width=2)
p.patch("px","py", source=auc_src, fill_color="red", fill_alpha=0.35, line_color=None)

# Invisible selection helpers; suppress selection visuals
p.scatter(
    "x","y", source=src, size=3,
    alpha=0.001, line_alpha=0.001,
    nonselection_fill_alpha=0.0, nonselection_line_alpha=0.0,
    selection_fill_alpha=0.0, selection_line_alpha=0.0,
    hover_fill_alpha=0.0, hover_line_alpha=0.0,
)

cb = CustomJS(args=dict(src=src, auc_src=auc_src), code="""
    // Get selection x-range
    const g = cb_obj.geometry || {};
    const x0 = g.x0, x1 = g.x1;
    // Clear previous fill
    auc_src.data = {px:[], py:[]};

    if (x0 == null || x1 == null) { auc_src.change.emit(); return; }
    const xmin = Math.min(x0, x1), xmax = Math.max(x0, x1);

    const x = src.data.x, y = src.data.y;
    // Indices inside selection
    const idx = [];
    for (let i=0;i<x.length;i++) if (x[i]>=xmin && x[i]<=xmax) idx.push(i);
    if (idx.length === 0) { auc_src.change.emit(); return; }

    // Tallest point within selection
    let best = -1, besty = -Infinity;
    for (const i of idx) { if (y[i] > besty) { besty = y[i]; best = i; } }
    if (best < 0) { auc_src.change.emit(); return; }

    // Grow peak region around apex: monotone up then down (simple robust rule)
    const eps = 1e-9;
    let L = best, R = best;
    while (L > 0 && y[L-1] <= y[L] + eps) L--;
    while (R < y.length-1 && y[R+1] <= y[R] + eps) R++;

    // Clamp L,R to selection window as well
    while (L < best && x[L] < xmin) L++;
    while (R > best && x[R] > xmax) R--;

    if (L >= R) { auc_src.change.emit(); return; }

    // Build polygon to baseline (y=0): [L..R] then back to L at y=0
    const px = [], py = [];
    for (let i=L;i<=R;i++) { px.push(x[i]); py.push(y[i]); }
    for (let i=R;i>=L;i--) { px.push(x[i]); py.push(0); }

    auc_src.data = {px, py};
    auc_src.change.emit();

    // Clear selection highlight immediately
    try { src.selected.indices = []; } catch (_) {}
""")

p.js_on_event(SelectionGeometry, cb)
show(p)


Loading BokehJS ...

In [13]:
import numpy as np, pandas as pd
import bokeh, bokeh.io
bokeh.io.output_notebook()

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.events import SelectionGeometry
from bokeh.palettes import Category20, Turbo256

# Example data dict: name -> DataFrame(time,intensity)
rng = np.random.default_rng(1)
def make_trace(mu, sig, amp=1.5):
    x = np.linspace(0, 30, 2000)
    y = amp*np.exp(-0.5*((x-mu)/sig)**2) + 0.05*rng.normal(size=x.size)
    return pd.DataFrame({"time": x, "intensity": y})

samples = {
    "s1": make_trace(8.5, 0.7, 1.6),
    "s2": make_trace(14.8, 1.0, 1.3),
    "s3": make_trace(22.0, 0.9, 1.8),
}

colors = list(Category20[20])[:len(samples)] if len(samples) <= 20 else [Turbo256[i] for i in np.linspace(0,255,len(samples),dtype=int)]

p = figure(
    title="Multi-sample: box select; AUC of tallest peak filled (red) per sample",
    height=360, width=980,
    x_axis_label="time", y_axis_label="intensity",
    tools="pan,wheel_zoom,box_zoom,box_select,reset,save,hover",
    active_drag="box_select",
)

# One src + one AUC patch per sample, one callback per sample
for i, (name, df) in enumerate(samples.items()):
    x = df["time"].to_numpy(); y = df["intensity"].to_numpy()
    src = ColumnDataSource(dict(x=x, y=y))
    auc_src = ColumnDataSource(dict(px=[], py=[]))

    p.line("x","y", source=src, color=colors[i%len(colors)], legend_label=name, line_width=2)
    p.patch("px","py", source=auc_src, fill_color="red", fill_alpha=0.35, line_color=None)

    p.scatter(
        "x","y", source=src, size=3,
        alpha=0.001, line_alpha=0.001,
        nonselection_fill_alpha=0.0, nonselection_line_alpha=0.0,
        selection_fill_alpha=0.0, selection_line_alpha=0.0,
        hover_fill_alpha=0.0, hover_line_alpha=0.0,
    )

    cb = CustomJS(args=dict(src=src, auc_src=auc_src), code="""
        const g = cb_obj.geometry || {};
        const x0 = g.x0, x1 = g.x1;
        auc_src.data = {px:[], py:[]};
        if (x0 == null || x1 == null) { auc_src.change.emit(); return; }
        const xmin = Math.min(x0, x1), xmax = Math.max(x0, x1);

        const x = src.data.x, y = src.data.y;
        const idx = [];
        for (let i=0;i<x.length;i++) if (x[i]>=xmin && x[i]<=xmax) idx.push(i);
        if (idx.length === 0) { auc_src.change.emit(); return; }

        let best = -1, besty = -Infinity;
        for (const i of idx) { if (y[i] > besty) { besty = y[i]; best = i; } }
        if (best < 0) { auc_src.change.emit(); return; }

        const eps = 1e-9;
        let L = best, R = best;
        while (L > 0 && y[L-1] <= y[L] + eps) L--;
        while (R < y.length-1 && y[R+1] <= y[R] + eps) R++;

        while (L < best && x[L] < xmin) L++;
        while (R > best && x[R] > xmax) R--;

        if (L >= R) { auc_src.change.emit(); return; }

        const px = [], py = [];
        for (let i=L;i<=R;i++) { px.push(x[i]); py.push(y[i]); }
        for (let i=R;i>=L;i--) { px.push(x[i]); py.push(0); }

        auc_src.data = {px, py};
        auc_src.change.emit();

        try { src.selected.indices = []; } catch(_) {}
    """)

    p.js_on_event(SelectionGeometry, cb)

p.legend.click_policy = "hide"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

show(p)


Loading BokehJS ...